In [4]:
import re
import requests
from dataclasses import dataclass
from textwrap import dedent

import nbformat as nbf
from bs4 import BeautifulSoup


In [5]:
WEEK_NB = 3


URLS = {
    "Roman Numerals Decoder": "https://www.codewars.com/kata/51b6249c4612257ac0000005/python",
    "Best Travel": "https://www.codewars.com/kata/55e7280b40e1c4a06d0000aa/python",
    "Array Diff": "https://www.codewars.com/kata/523f5d21c841566fde000009/python",
    "Directions Reduction": "https://www.codewars.com/kata/550f22f4d758534c1100025a/python",
    "Make the Deadfish swim": "https://www.codewars.com/kata/51e0007c1f9378fa810002a9/python",
    "Is the meetup age-diverse? (High-Order Func)": "https://www.codewars.com/kata/5829ca646d02cd1a65000284/python",
}

In [6]:
# for title, url in URLS.items():
#     response = requests.get(url)
#     print(f"---{title}---")
#     ranks = list(re.findall(r'(rank="-\d+"|\d+ kyu)', response.content.decode()))
#     print(ranks)

In [7]:
@dataclass
class Challenge:
    title: str
    level: int
    descr: str

def get_challenge(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error fetching page: {response}")

    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.title.text.replace(" | Codewars", "")

    content = response.content.decode()
    m = re.search(r'\\"description\\":\\"(.*?)\\"', content, flags=re.S)
    descr = m.group(1).replace("\\\\n", "\n")

    # there might be some content specific to code: ```if:<language> or ```if-not:<language>
    descr = re.sub(r"^```if:(?!python)\w+.*?^```", "", descr, flags=re.S|re.M)
    descr = re.sub(r"^```if-not:python.*?^```", "", descr, flags=re.S|re.M)

    level = re.search(r"(\d) kyu", content).group(1)
    
    return Challenge(title, int(level), descr)


In [8]:
# print(get_description("https://www.codewars.com/kata/5a2fd38b55519ed98f0000ce/python"))

In [11]:
LEVELS = {
    8:"8️⃣",
    7:"7️⃣",
    6:"6️⃣",
    5:"5️⃣",
    4:"4️⃣",
    3:"3️⃣",
    2:"2️⃣",
    1:"1️⃣",
}


def iter_challenges(week_nb, urls):
    header = dedent(f"""\
        # ⇝ Coding 💻 Challenges Week {week_nb} ⇜

        ![unpackAI Logo](../images/unpackAI_logo_whiteBG.svg)
            
            
        **NOTE**: in the solution, the code assert function(...) == value is used to test the solution.
        ... so you don't have to care about it
    """)
    yield nbf.v4.new_markdown_cell(header)

    for url in urls:
        challenge = get_challenge(url)
        print(f"* {challenge.title} ({challenge.level})")
        train_url = re.sub(f"(/train)?/python$", "", url) + "/train/python"

        title = dedent(f"""\
        # {challenge.title} {LEVELS[challenge.level]}
        🔗{train_url}
        """)
        yield nbf.v4.new_markdown_cell(f"{title}\n{challenge.descr}")

        yield nbf.v4.new_code_cell("# TODO: add solution")


In [12]:
fname = f'challenges_W{WEEK_NB:02d}_TMP.ipynb'
print(f"Starting creation of Notebook {fname}:")

notebook = nbf.v4.new_notebook()
notebook['cells'] = list(iter_challenges(WEEK_NB, URLS.values()))

with open(fname, 'w', encoding="utf-8") as f:
    nbf.write(notebook, f)
print("=> SUCCESSFUL!")

Starting creation of Notebook challenges_W03_TMP.ipynb:
* Roman Numerals Decoder (6)
* Best travel (5)
* Array.diff (6)
* Directions Reduction (5)
* Make the Deadfish swim (6)
* Coding Meetup #9 - Higher-Order Functions Series - Is the meetup age-diverse? (6)
=> SUCCESSFUL!
